<a href="https://colab.research.google.com/github/neemiasbsilva/MLLMs-Teoria-e-Pratica/blob/main/use-cases/IdentifyFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepSeek VL - Identificação de objetos

Bem vindos a esse notebook com um pequeno tutorial para identificação de objetos em imagens utilizando uma LLM multimodal!

É muito importante que cada célula seja executada em sequência, conforme será explicado em logo mais.

Neste tutorial, devido às limitações do Google Colab, vamos utilizar o [Deepseek-VL](https://github.com/deepseek-ai/DeepSeek-VL). Então, Vamos começar clonando seu repositório:

## Configuração do Ambiente

In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL -q

Em seguida, vamos instalar as dependências, tomando cuidado para instalar o Torch separado, já que este é sensível a versão que estamos utilizando de Python:

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
!sed -i '/torch==/d' /content/DeepSeek-VL/requirements.txt
!pip install -r /content/DeepSeek-VL/requirements.txt -q

Após a instalação precisa reinicar a execução para que os pacotes necessários sejam devidamente instalados no kernel

## Carregar o Modelo e Inicialização do Tokenizer

Agora, vamos utilizar os métodos da biblioteca transformers e o torch para carregar o modelo na nossa placa gráfica, inicializar nosso tokenizer, que será responsável pela codificação das entradas, e decodificação das saídas para o modelo:

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/content/DeepSeek-VL')

import torch
from transformers import AutoModelForCausalLM

from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM

model_path = "deepseek-ai/deepseek-vl-1.3b-base"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

## Carregando o Modelo e Fazendo um Teste de Infêrencia

Vamos rodar este teste para verificar se o modelo está carregado corretamente! Note a reposta sendo mostrada no terminal, e todo o pré-processamento que um modelo multimodal exige, como criação dos embeddings das imagens:

In [ ]:
from deepseek_vl.utils.io import load_pil_images
from IPython.display import Image, display

conversation = [
    {
        "role": "User",
        "content": """<image_placeholder>Analyze the image and identify which of the features you are certain are in this list and in the image: [dog, cat, rat, bird, sheep]. Do not mention features not in the list.
        """,
        "images": ["/content/DeepSeek-VL/images/dog_a.png"]
    },
    {
        "role": "Assistant",
        "content": ""
    }
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

display(Image("/content/DeepSeek-VL/images/dog_a.png"))

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

## Inferência em Batch

Caso dê tudo certo, vamos agora tentar um processamento em batch.

Como já foi detalhado no material, vamos focar na task 1. então focamos puramente em passar a imagem, e um prompt para o modelo.

Neste exemplo, utilizamos o subconjunto alpha5 p3, onde todos os 5 examinadores concordaram no sentimento de uma imagem, com três classes.

Mais detalhes, e acesso para essa base de dados completa podem ser acessados [aqui](https://arxiv.org/abs/2508.16873)

In [ ]:
import pandas as pd
from io import BytesIO
base_url = "https://drive.usercontent.google.com/download?export=view&id="
ids = ["1mKb_EHSAgsOmxJsTFleVUvtGv4Uj8KQ0", "1UHIfdHUFkOsamfVcL_QwAG7TFc_gNwqk", "1QcaUZ0Ac-XsUmZvv-sIZr8x1waK7gnwL", "1o3a37CSBkb8uuPbg7xEItMc-47ZksmNl", "1_cDlBPgjCkokLs3xSLihxAys6WuWIDnW"]


Depois de carregado nosso conjunto, vamos passar por cada uma das imagens, passando para o modelo a imagem, e o nosso prompt, propriamente preparado, e mostrar a reposta dele! Note que para esse modelo, extrair o sentimento exato da resposta não é uma tarefa simples, mas em modelos mais avançados, essa tarefa se torna um pouco mais fácil, já que tendem a dar uma resposta mais assertiva.

In [ ]:
import requests
import base64

for id in ids:
  image_url = base_url + id
  response = requests.get(image_url)
  b64response = uri = ("data:" +
                response.headers['Content-Type'] + ";" +
                "base64," + str(base64.b64encode(response.content).decode("utf-8")))

  conversation = [
    {
        "role": "User",
        "content": """<image_placeholder>Analyze the image and provide a list of features that you are certain are present in the image. Ignore features not in the provided list.

        PROVIDED LIST:
        [sidewalk, tree-lined street, porch, fenced front yard, attached garage, cul-de-sac, hills, private front entrance]

        EXAMPLE OUTPUT:
        [sidewalk, porch, attached garage]
        """,
        "images": [b64response]
    },
    {
        "role": "Assistant",
        "content": ""
    }
  ]

  pil_images = load_pil_images(conversation)
  prepare_inputs = vl_chat_processor(
      conversations=conversation,
      images=pil_images,
      force_batchify=True
  ).to(vl_gpt.device)

  inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

  outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
  )

  answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
  display(Image(data = response.content))
  print('--- id: ' + id + '---')
  print(f"{prepare_inputs['sft_format'][0]}", answer)
  print('---------')


